In [1]:
import pandas as pd

In [2]:
player_df = pd.read_csv('../../data/player_stat_2022.csv')
player_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,42200116,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,F,NaN,39.000000:35,...,0.0,5.0,5.0,2.0,0.0,1.0,2.0,3.0,32.0,7.0
1,42200116,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,NaN,39.000000:52,...,2.0,12.0,14.0,7.0,1.0,2.0,0.0,1.0,30.0,11.0
2,42200116,1610612738,BOS,Boston,201143,Al Horford,Al,C,NaN,29.000000:54,...,3.0,9.0,12.0,4.0,2.0,3.0,0.0,4.0,10.0,9.0
3,42200116,1610612738,BOS,Boston,1628401,Derrick White,Derrick,G,NaN,29.000000:46,...,1.0,1.0,2.0,2.0,0.0,1.0,1.0,3.0,7.0,-1.0
4,42200116,1610612738,BOS,Boston,203935,Marcus Smart,Marcus,G,NaN,32.000000:33,...,3.0,1.0,4.0,4.0,1.0,0.0,2.0,0.0,22.0,11.0


In [3]:
player_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS'],
      dtype='object')

In [4]:
player_df.dtypes

GAME_ID                int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_CITY             object
PLAYER_ID              int64
PLAYER_NAME           object
NICKNAME              object
START_POSITION        object
COMMENT               object
MIN                   object
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TO                   float64
PF                   float64
PTS                  float64
PLUS_MINUS           float64
dtype: object

In [4]:
# Convert MIN column to float
def convert_minutes_to_float(time_str):
    if ':' in time_str:
        minutes, seconds = time_str.split(':')
        total_minutes = float(minutes) + float(seconds) / 60
    else:
        total_minutes = float(time_str)
    return total_minutes

# Convert MIN column to string type (if not already)
player_df['MIN'] = player_df['MIN'].astype(str)

# Apply the conversion function
player_df['MIN'] = player_df['MIN'].apply(convert_minutes_to_float)

player_df['MIN'].head()

0    39.583333
1    39.866667
2    29.900000
3    29.766667
4    32.550000
Name: MIN, dtype: float64

In [5]:
game_df = pd.read_csv('../../data/game_2022.csv')
# cleaning, game_df is unique on team_id and game_id level after fixing
game_df = game_df.dropna(subset=['WL'])
game_df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42022,1610612737,ATL,Atlanta Hawks,42200116,2023-04-27,ATL vs. BOS,L,241,120,...,0.773,12.0,33.0,45.0,28,5.0,7,10,15,-8.0
1,42022,1610612737,ATL,Atlanta Hawks,42200115,2023-04-25,ATL @ BOS,W,242,119,...,1.000,6.0,28.0,34.0,26,5.0,4,8,16,2.0
2,42022,1610612737,ATL,Atlanta Hawks,42200114,2023-04-23,ATL vs. BOS,L,240,121,...,0.875,11.0,31.0,42.0,25,8.0,4,12,24,-8.0
3,42022,1610612737,ATL,Atlanta Hawks,42200113,2023-04-21,ATL vs. BOS,W,240,130,...,0.813,11.0,37.0,48.0,24,5.0,6,18,15,8.0
4,42022,1610612737,ATL,Atlanta Hawks,42200112,2023-04-18,ATL @ BOS,L,241,106,...,0.500,19.0,30.0,49.0,21,10.0,4,15,11,-13.0


In [6]:
player_df.shape

(39273, 29)

In [8]:
game_df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [9]:
game_df.dtypes

SEASON_ID              int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID                int64
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                 float64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                    int64
STL                  float64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

In [7]:
# Change WL to numeric values
game_df['WL'] = game_df['WL'].map({'W': 1, 'L': 0})

In [8]:
game_df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42022,1610612737,ATL,Atlanta Hawks,42200116,2023-04-27,ATL vs. BOS,0,241,120,...,0.773,12.0,33.0,45.0,28,5.0,7,10,15,-8.0
1,42022,1610612737,ATL,Atlanta Hawks,42200115,2023-04-25,ATL @ BOS,1,242,119,...,1.000,6.0,28.0,34.0,26,5.0,4,8,16,2.0
2,42022,1610612737,ATL,Atlanta Hawks,42200114,2023-04-23,ATL vs. BOS,0,240,121,...,0.875,11.0,31.0,42.0,25,8.0,4,12,24,-8.0
3,42022,1610612737,ATL,Atlanta Hawks,42200113,2023-04-21,ATL vs. BOS,1,240,130,...,0.813,11.0,37.0,48.0,24,5.0,6,18,15,8.0
4,42022,1610612737,ATL,Atlanta Hawks,42200112,2023-04-18,ATL @ BOS,0,241,106,...,0.500,19.0,30.0,49.0,21,10.0,4,15,11,-13.0


In [9]:
# Convert GAME_DATE to datetime
game_df['GAME_DATE'] = pd.to_datetime(game_df['GAME_DATE'])

# Sort the DataFrame by player and date
game_df.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'], inplace=True)
game_df.reset_index(drop=True, inplace=True)

# Define a window size for calculating recent averages
window_sizes = [1,3,5,10,15,30]  # You can adjust this as needed

# Calculate previous games rolling averages of team statistics
rolling_cols = ['WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
for col in rolling_cols:
    for window in window_sizes:
        game_df[f'{col}_avg_{window}'] = game_df.groupby('TEAM_ABBREVIATION')[col].shift(1).rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

game_df.head(20)

/var/folders/b6/nt5dw2rs5w72ym11_8rbv87c0000gn/T/ipykernel_81453/1827983145.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_df[f'{col}_avg_{window}'] = game_df.groupby('TEAM_ABBREVIATION')[col].shift(1).rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
/var/folders/b6/nt5dw2rs5w72ym11_8rbv87c0000gn/T/ipykernel_81453/1827983145.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_df[f'{col}_avg_{window}'] = game_df.groupby('TEAM_ABBREVIATION')[col].shift(1).rolling(window=window, min_peri

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,TOV_avg_5,TOV_avg_10,TOV_avg_15,TOV_avg_30,PF_avg_1,PF_avg_3,PF_avg_5,PF_avg_10,PF_avg_15,PF_avg_30
0,22022,1610612737,ATL,Atlanta Hawks,1522200014,2022-07-09,ATL @ UTA,0,200,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22022,1610612737,ATL,Atlanta Hawks,1522200026,2022-07-11,ATL vs. NOP,0,200,73,...,12.000000,12.000000,12.000000,12.000000,23.0,23.000000,23.000000,23.000000,23.000000,23.000000
2,22022,1610612737,ATL,Atlanta Hawks,1522200034,2022-07-12,ATL @ MIA,1,199,95,...,11.500000,11.500000,11.500000,11.500000,24.0,23.500000,23.500000,23.500000,23.500000,23.500000
3,22022,1610612737,ATL,Atlanta Hawks,1522200046,2022-07-14,ATL @ SAS,1,199,87,...,12.666667,12.666667,12.666667,12.666667,29.0,25.333333,25.333333,25.333333,25.333333,25.333333
4,22022,1610612737,ATL,Atlanta Hawks,1522200061,2022-07-16,ATL @ CLE,0,199,90,...,12.750000,12.750000,12.750000,12.750000,19.0,24.000000,23.750000,23.750000,23.750000,23.750000
5,12022,1610612737,ATL,Atlanta Hawks,12200023,2022-10-06,ATL vs. MIL,1,241,123,...,12.000000,12.000000,12.000000,12.000000,19.0,22.333333,22.800000,22.800000,22.800000,22.800000
6,12022,1610612737,ATL,Atlanta Hawks,12200036,2022-10-08,ATL @ MIL,1,240,118,...,11.000000,11.166667,11.166667,11.166667,17.0,18.333333,21.600000,21.833333,21.833333,21.833333
7,12022,1610612737,ATL,Atlanta Hawks,12200052,2022-10-12,ATL @ CLE,0,238,99,...,11.000000,11.142857,11.142857,11.142857,26.0,20.666667,22.000000,22.428571,22.428571,22.428571
8,12022,1610612737,ATL,Atlanta Hawks,12200067,2022-10-14,ATL @ NOP,0,240,111,...,11.200000,11.750000,11.750000,11.750000,15.0,19.333333,19.200000,21.500000,21.500000,21.500000
9,22022,1610612737,ATL,Atlanta Hawks,22200005,2022-10-19,ATL vs. HOU,1,240,117,...,11.400000,12.000000,12.000000,12.000000,23.0,21.333333,20.000000,21.666667,21.666667,21.666667


In [10]:
game_df.shape

(2949, 142)

In [12]:
# Merge player_df with game_df to get the date of each game
merged_df = player_df.merge(game_df[['GAME_ID', 'TEAM_ID','GAME_DATE']], on=['GAME_ID', 'TEAM_ID'], how='left')
# Convert GAME_DATE to datetime
merged_df['GAME_DATE'] = pd.to_datetime(merged_df['GAME_DATE'])
merged_df.shape

(39273, 30)

In [13]:
# Sort the DataFrame by player and date
merged_df.sort_values(['PLAYER_ID', 'GAME_DATE'], inplace=True)
merged_df.reset_index(drop=True, inplace=True)

# Define a window size for calculating recent averages
window_sizes = [1,3,5,10,15,30]  # You can adjust this as needed

# Calculate rolling averages for various statistics
rolling_cols = ['MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']
for col in rolling_cols:
    for window in window_sizes:
        merged_df[f'{col}_avg_{window}'] = merged_df.groupby('PLAYER_ID')[col].shift(1).rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

merged_df[['NICKNAME', 'GAME_DATE', 'COMMENT', 'PTS', 'PTS_avg_3', 'PTS_avg_5', 'PTS_avg_10']].head(20)

/var/folders/b6/nt5dw2rs5w72ym11_8rbv87c0000gn/T/ipykernel_81453/3982283650.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[f'{col}_avg_{window}'] = merged_df.groupby('PLAYER_ID')[col].shift(1).rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)


,NICKNAME,GAME_DATE,COMMENT,PTS,PTS_avg_3,PTS_avg_5,PTS_avg_10
0,LeBron,2022-10-03,NaN,4.0,NaN,NaN,NaN
1,LeBron,2022-10-05,NaN,23.0,4.000000,4.000000,4.000000
2,LeBron,2022-10-06,DNP - Coach's Decision,NaN,13.500000,13.500000,13.500000
3,LeBron,2022-10-09,DNP - Coach's Decision,NaN,13.500000,13.500000,13.500000
4,LeBron,2022-10-12,NaN,25.0,23.000000,13.500000,13.500000
5,LeBron,2022-10-14,NaN,10.0,25.000000,17.333333,17.333333
6,LeBron,2022-10-18,NaN,31.0,17.500000,19.333333,15.500000
7,LeBron,2022-10-20,NaN,20.0,22.000000,22.000000,18.600000
8,LeBron,2022-10-23,NaN,31.0,20.333333,21.500000,18.833333
9,LeBron,2022-10-26,NaN,19.0,27.333333,23.400000,20.571429


In [14]:
merged_df.shape

(39273, 126)

In [15]:
merged_df = merged_df.merge(game_df, on=['GAME_ID', 'TEAM_ID'], how='left', suffixes=(None, '_TEAM'))
merged_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TOV_avg_5,TOV_avg_10,TOV_avg_15,TOV_avg_30,PF_avg_1_TEAM,PF_avg_3_TEAM,PF_avg_5_TEAM,PF_avg_10_TEAM,PF_avg_15_TEAM,PF_avg_30_TEAM
0,12200012,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,15.683333,...,12.2,13.333333,13.428571,13.275862,20.0,20.000000,20.8,20.666667,20.928571,19.827586
1,12200022,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.433333,...,14.2,14.111111,14.000000,13.724138,20.0,18.666667,20.0,20.444444,21.142857,19.689655
2,12200028,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,14.4,14.300000,14.428571,13.689655,25.0,21.666667,21.0,20.900000,21.642857,19.965517
3,12200040,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,15.4,14.700000,14.785714,13.724138,22.0,22.333333,20.6,21.300000,21.714286,20.103448
4,12200057,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,24.700000,...,16.4,14.800000,14.928571,14.034483,26.0,24.333333,22.6,21.700000,22.142857,20.310345


In [16]:
# Merge the dataframes based on game_id and TEAM_ID mismatch
merged_df = merged_df.merge(game_df, on='GAME_ID', how='left', suffixes=(None, '_OPP'))
merged_df = merged_df[~merged_df['TEAM_ID'].eq(merged_df['TEAM_ID_OPP'])]
merged_df.shape
# Now merged_df contains the rows where game_id matches but TEAM_ID is different

(39183, 407)

In [17]:
merged_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TOV_avg_5_OPP,TOV_avg_10_OPP,TOV_avg_15_OPP,TOV_avg_30_OPP,PF_avg_1_OPP,PF_avg_3_OPP,PF_avg_5_OPP,PF_avg_10_OPP,PF_avg_15_OPP,PF_avg_30_OPP
1,12200012,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,15.683333,...,16.6,15.222222,14.428571,14.000000,14.0,20.000000,22.0,21.777778,20.857143,19.758621
3,12200022,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.433333,...,15.6,13.555556,12.428571,12.758621,17.0,19.666667,21.0,21.666667,20.571429,20.241379
5,12200028,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,15.8,14.888889,14.857143,13.862069,16.0,20.333333,21.4,19.777778,18.571429,17.586207
6,12200040,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,17.8,17.500000,16.571429,15.586207,25.0,24.000000,23.6,23.200000,23.214286,22.068966
9,12200057,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,24.700000,...,15.6,15.000000,14.857143,13.931034,18.0,21.666667,22.0,21.777778,19.785714,18.137931


In [18]:
merged_df.to_csv('../../data/player_stat_team_stat_aggregate.csv', index = False)

In [19]:
merged_df.to_parquet('../../data/player_stat_team_stat_aggregate.parquet')